In [4]:
import sqlalchemy

#Connect a database
from sqlalchemy import create_engine
engine = create_engine('sqlite:///ex.db', echo = True)

In [5]:
from sqlalchemy.ext.declarative import declarative_base
Base = declarative_base()

In [6]:
from sqlalchemy import Column, Integer, String

class User(Base):
    __tablename__ = 'users'
    
    id = Column(Integer, primary_key=True)
    name = Column(String)
    fullname = Column(String)
    password = Column(String)
    
    def __repr__(self):
        return "<User(name='%s', fullname='%s', password='%s')>" % (
            self.name, self.fullname, self.password
        )

In [7]:
from sqlalchemy.orm import sessionmaker

Session = sessionmaker(bind=engine)
session = Session()

In [8]:
Base.metadata.create_all(engine)

2022-12-15 18:09:29,993 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-12-15 18:09:29,995 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("users")
2022-12-15 18:09:29,996 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-12-15 18:09:29,997 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("users")
2022-12-15 18:09:30,000 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-12-15 18:09:30,004 INFO sqlalchemy.engine.Engine 
CREATE TABLE users (
	id INTEGER NOT NULL, 
	name VARCHAR, 
	fullname VARCHAR, 
	password VARCHAR, 
	PRIMARY KEY (id)
)


2022-12-15 18:09:30,029 INFO sqlalchemy.engine.Engine [no key 0.00231s] ()
2022-12-15 18:09:30,037 INFO sqlalchemy.engine.Engine COMMIT


In [9]:
patricia = User(name='Patricia', fullname='Patricia Figueroa Millán', password='lifeonmars')
session.add(patricia)
patricia

<User(name='Patricia', fullname='Patricia Figueroa Millán', password='lifeonmars')>

In [10]:
user = session.query(User).filter_by(name='Patricia').first()

2022-12-15 18:09:37,104 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-12-15 18:09:37,109 INFO sqlalchemy.engine.Engine INSERT INTO users (name, fullname, password) VALUES (?, ?, ?)
2022-12-15 18:09:37,111 INFO sqlalchemy.engine.Engine [generated in 0.00188s] ('Patricia', 'Patricia Figueroa Millán', 'lifeonmars')
2022-12-15 18:09:37,138 INFO sqlalchemy.engine.Engine SELECT users.id AS users_id, users.name AS users_name, users.fullname AS users_fullname, users.password AS users_password 
FROM users 
WHERE users.name = ?
 LIMIT ? OFFSET ?
2022-12-15 18:09:37,146 INFO sqlalchemy.engine.Engine [generated in 0.00328s] ('Patricia', 1, 0)


In [11]:
session.add_all([
    User(name='Pedro', fullname='Pedro Álvarez', password='12345'),
    User(name='Pablo', fullname='Pablo Alcaraz', password='67890'),
    User(name='Penélope', fullname='Penélope Sánchez', password='09876')
])

patricia.password = '3121077428'

In [12]:
session.commit()

2022-12-15 18:09:43,155 INFO sqlalchemy.engine.Engine UPDATE users SET password=? WHERE users.id = ?
2022-12-15 18:09:43,158 INFO sqlalchemy.engine.Engine [generated in 0.00190s] ('3121077428', 1)
2022-12-15 18:09:43,159 INFO sqlalchemy.engine.Engine INSERT INTO users (name, fullname, password) VALUES (?, ?, ?)
2022-12-15 18:09:43,161 INFO sqlalchemy.engine.Engine [cached since 6.052s ago] ('Pedro', 'Pedro Álvarez', '12345')
2022-12-15 18:09:43,163 INFO sqlalchemy.engine.Engine INSERT INTO users (name, fullname, password) VALUES (?, ?, ?)
2022-12-15 18:09:43,166 INFO sqlalchemy.engine.Engine [cached since 6.056s ago] ('Pablo', 'Pablo Alcaraz', '67890')
2022-12-15 18:09:43,166 INFO sqlalchemy.engine.Engine INSERT INTO users (name, fullname, password) VALUES (?, ?, ?)
2022-12-15 18:09:43,169 INFO sqlalchemy.engine.Engine [cached since 6.06s ago] ('Penélope', 'Penélope Sánchez', '09876')
2022-12-15 18:09:43,171 INFO sqlalchemy.engine.Engine COMMIT


In [13]:
patricia.name = 'Eduardo'

fake_user = User(name='fakeuser', fullname='Invalid', password='12345')
session.add(fake_user)

session.query(User).filter(User.name.in_(['Eduardo', 'fakeuser'])).all()

2022-12-15 18:09:46,538 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-12-15 18:09:46,546 INFO sqlalchemy.engine.Engine SELECT users.id AS users_id, users.fullname AS users_fullname, users.password AS users_password 
FROM users 
WHERE users.id = ?
2022-12-15 18:09:46,546 INFO sqlalchemy.engine.Engine [generated in 0.00241s] (1,)
2022-12-15 18:09:46,555 INFO sqlalchemy.engine.Engine UPDATE users SET name=? WHERE users.id = ?
2022-12-15 18:09:46,555 INFO sqlalchemy.engine.Engine [generated in 0.00142s] ('Eduardo', 1)
2022-12-15 18:09:46,555 INFO sqlalchemy.engine.Engine INSERT INTO users (name, fullname, password) VALUES (?, ?, ?)
2022-12-15 18:09:46,555 INFO sqlalchemy.engine.Engine [cached since 9.453s ago] ('fakeuser', 'Invalid', '12345')
2022-12-15 18:09:46,563 INFO sqlalchemy.engine.Engine SELECT users.id AS users_id, users.name AS users_name, users.fullname AS users_fullname, users.password AS users_password 
FROM users 
WHERE users.name IN (?, ?)
2022-12-15 18:09:46,563 INFO 

[<User(name='Eduardo', fullname='Patricia Figueroa Millán', password='3121077428')>,
 <User(name='fakeuser', fullname='Invalid', password='12345')>]

In [14]:
session.rollback()

2022-12-15 18:09:50,768 INFO sqlalchemy.engine.Engine ROLLBACK


In [15]:
patricia.name

2022-12-15 18:09:54,037 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-12-15 18:09:54,045 INFO sqlalchemy.engine.Engine SELECT users.id AS users_id, users.name AS users_name, users.fullname AS users_fullname, users.password AS users_password 
FROM users 
WHERE users.id = ?
2022-12-15 18:09:54,045 INFO sqlalchemy.engine.Engine [generated in 0.00216s] (1,)


'Patricia'

In [16]:
fake_user in session

False

In [17]:
for instance in session.query(User).order_by(User.id):
    print(instance)

2022-12-15 18:09:59,684 INFO sqlalchemy.engine.Engine SELECT users.id AS users_id, users.name AS users_name, users.fullname AS users_fullname, users.password AS users_password 
FROM users ORDER BY users.id
2022-12-15 18:09:59,686 INFO sqlalchemy.engine.Engine [generated in 0.00219s] ()
<User(name='Patricia', fullname='Patricia Figueroa Millán', password='3121077428')>
<User(name='Pedro', fullname='Pedro Álvarez', password='12345')>
<User(name='Pablo', fullname='Pablo Alcaraz', password='67890')>
<User(name='Penélope', fullname='Penélope Sánchez', password='09876')>


In [18]:
for name, fullname in session.query(User.name, User.fullname):
    print(name, fullname)

2022-12-15 18:10:02,552 INFO sqlalchemy.engine.Engine SELECT users.name AS users_name, users.fullname AS users_fullname 
FROM users
2022-12-15 18:10:02,555 INFO sqlalchemy.engine.Engine [generated in 0.00239s] ()
Patricia Patricia Figueroa Millán
Pedro Pedro Álvarez
Pablo Pablo Alcaraz
Penélope Penélope Sánchez


In [19]:
for user in session.query(User).filter(User.name == 'Patricia'):
    print(user)

2022-12-15 18:10:05,301 INFO sqlalchemy.engine.Engine SELECT users.id AS users_id, users.name AS users_name, users.fullname AS users_fullname, users.password AS users_password 
FROM users 
WHERE users.name = ?
2022-12-15 18:10:05,301 INFO sqlalchemy.engine.Engine [generated in 0.00337s] ('Patricia',)
<User(name='Patricia', fullname='Patricia Figueroa Millán', password='3121077428')>


In [20]:
for user in session.query(User).filter(User.name != 'Patricia'):
    print(user)

2022-12-15 18:10:09,167 INFO sqlalchemy.engine.Engine SELECT users.id AS users_id, users.name AS users_name, users.fullname AS users_fullname, users.password AS users_password 
FROM users 
WHERE users.name != ?
2022-12-15 18:10:09,167 INFO sqlalchemy.engine.Engine [generated in 0.00162s] ('Patricia',)
<User(name='Pedro', fullname='Pedro Álvarez', password='12345')>
<User(name='Pablo', fullname='Pablo Alcaraz', password='67890')>
<User(name='Penélope', fullname='Penélope Sánchez', password='09876')>


In [21]:
for user in session.query(User).filter(User.name.like('Patricia%')):
    print(user)

2022-12-15 18:10:11,438 INFO sqlalchemy.engine.Engine SELECT users.id AS users_id, users.name AS users_name, users.fullname AS users_fullname, users.password AS users_password 
FROM users 
WHERE users.name LIKE ?
2022-12-15 18:10:11,446 INFO sqlalchemy.engine.Engine [generated in 0.00132s] ('Patricia%',)
<User(name='Patricia', fullname='Patricia Figueroa Millán', password='3121077428')>


In [22]:
for user in session.query(User).filter(User.name.in_(['Patricia', 'Pablo', 'Pedro'])):
    print(user)

2022-12-15 18:10:14,848 INFO sqlalchemy.engine.Engine SELECT users.id AS users_id, users.name AS users_name, users.fullname AS users_fullname, users.password AS users_password 
FROM users 
WHERE users.name IN (?, ?, ?)
2022-12-15 18:10:14,850 INFO sqlalchemy.engine.Engine [cached since 28.29s ago] ('Patricia', 'Pablo', 'Pedro')
<User(name='Patricia', fullname='Patricia Figueroa Millán', password='3121077428')>
<User(name='Pedro', fullname='Pedro Álvarez', password='12345')>
<User(name='Pablo', fullname='Pablo Alcaraz', password='67890')>


In [23]:
for user in session.query(User).filter(~User.name.in_(['Patricia', 'Pablo', 'Pedro'])):
    print(user)

2022-12-15 18:10:17,851 INFO sqlalchemy.engine.Engine SELECT users.id AS users_id, users.name AS users_name, users.fullname AS users_fullname, users.password AS users_password 
FROM users 
WHERE (users.name NOT IN (?, ?, ?))
2022-12-15 18:10:17,851 INFO sqlalchemy.engine.Engine [generated in 0.00210s] ('Patricia', 'Pablo', 'Pedro')
<User(name='Penélope', fullname='Penélope Sánchez', password='09876')>


In [24]:
for user in session.query(User).filter(User.name == 'Patricia', User.fullname == 'Patricia Figueroa Millán'):
    print(user)

2022-12-15 18:10:20,110 INFO sqlalchemy.engine.Engine SELECT users.id AS users_id, users.name AS users_name, users.fullname AS users_fullname, users.password AS users_password 
FROM users 
WHERE users.name = ? AND users.fullname = ?
2022-12-15 18:10:20,118 INFO sqlalchemy.engine.Engine [generated in 0.00201s] ('Patricia', 'Patricia Figueroa Millán')
<User(name='Patricia', fullname='Patricia Figueroa Millán', password='3121077428')>


In [25]:
from sqlalchemy import or_
for user in session.query(User).filter(or_(User.name == 'Patricia', User.name == 'Pablo')):
    print(user)

2022-12-15 18:10:22,493 INFO sqlalchemy.engine.Engine SELECT users.id AS users_id, users.name AS users_name, users.fullname AS users_fullname, users.password AS users_password 
FROM users 
WHERE users.name = ? OR users.name = ?
2022-12-15 18:10:22,493 INFO sqlalchemy.engine.Engine [generated in 0.00154s] ('Patricia', 'Pablo')
<User(name='Patricia', fullname='Patricia Figueroa Millán', password='3121077428')>
<User(name='Pablo', fullname='Pablo Alcaraz', password='67890')>


In [26]:
from sqlalchemy import ForeignKey
from sqlalchemy.orm import relationship

class Address(Base):
    __tablename__ = 'addresses'
    id = Column(Integer, primary_key=True)
    email_address = Column(String, nullable=False)
    user_id = Column(Integer, ForeignKey('users.id'))
    user = relationship("User", back_populates="addresses")
    
    def __repr__(self):
        return "<Address(email_address='%s')>" % self.email_address

In [27]:
User.addresses = relationship("Address", order_by=Address.id, back_populates="user")

Base.metadata.create_all(engine)

2022-12-15 18:10:28,413 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-12-15 18:10:28,415 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("users")
2022-12-15 18:10:28,416 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-12-15 18:10:28,418 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("addresses")
2022-12-15 18:10:28,420 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-12-15 18:10:28,423 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("addresses")
2022-12-15 18:10:28,425 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-12-15 18:10:28,427 INFO sqlalchemy.engine.Engine 
CREATE TABLE addresses (
	id INTEGER NOT NULL, 
	email_address VARCHAR NOT NULL, 
	user_id INTEGER, 
	PRIMARY KEY (id), 
	FOREIGN KEY(user_id) REFERENCES users (id)
)


2022-12-15 18:10:28,428 INFO sqlalchemy.engine.Engine [no key 0.00130s] ()
2022-12-15 18:10:28,443 INFO sqlalchemy.engine.Engine COMMIT


In [28]:
jack = User(name='jack', fullname='Jack Bean', password='gjffdd')
jack.addresses

[]

In [29]:
jack.addresses = [Address(email_address='jack@google.com'), Address(email_address='j25@yahoo.com')]

In [33]:
jack.addresses[1]

<Address(email_address='j25@yahoo.com')>

In [31]:
jack.addresses[1].user

<User(name='jack', fullname='Jack Bean', password='gjffdd')>

In [32]:
session.add(jack)

session.commit()

jack = session.query(User).filter_by(name='jack').one()
jack.addresses 

2022-12-15 18:11:10,496 INFO sqlalchemy.engine.Engine INSERT INTO users (name, fullname, password) VALUES (?, ?, ?)
2022-12-15 18:11:10,504 INFO sqlalchemy.engine.Engine [generated in 0.00223s] ('jack', 'Jack Bean', 'gjffdd')
2022-12-15 18:11:10,540 INFO sqlalchemy.engine.Engine INSERT INTO addresses (email_address, user_id) VALUES (?, ?)
2022-12-15 18:11:10,548 INFO sqlalchemy.engine.Engine [generated in 0.00633s] ('jack@google.com', 5)
2022-12-15 18:11:10,548 INFO sqlalchemy.engine.Engine INSERT INTO addresses (email_address, user_id) VALUES (?, ?)
2022-12-15 18:11:10,548 INFO sqlalchemy.engine.Engine [cached since 0.01357s ago] ('j25@yahoo.com', 5)
2022-12-15 18:11:10,556 INFO sqlalchemy.engine.Engine COMMIT
2022-12-15 18:11:10,564 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-12-15 18:11:10,572 INFO sqlalchemy.engine.Engine SELECT users.id AS users_id, users.name AS users_name, users.fullname AS users_fullname, users.password AS users_password 
FROM users 
WHERE users.name = 

[<Address(email_address='jack@google.com')>,
 <Address(email_address='j25@yahoo.com')>]